In [4]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, Flatten, concatenate, Lambda, BatchNormalization, Dropout
from tensorflow.keras.models import Model
import pandas as pd
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
import numpy as np
import tensorflow_hub as hub
import cv2
import csv
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.utils.vis_utils import plot_model
import tensorflow.keras.backend as K

In [5]:
random_number = 42

In [6]:
tf.keras.utils.set_random_seed(random_number)

In [7]:
images_dir = 'C:/Users/prokopj/resized/'

In [8]:
pairs = pd.read_csv("pairs2000.csv")

FileNotFoundError: ignored

In [ ]:
pairs['Path 1'] = images_dir + pairs['Path 1']
pairs['Path 2'] = images_dir + pairs['Path 2']

In [ ]:
pairs["Label"].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(pairs, stratify=pairs["Label"], test_size=0.1, shuffle = True, random_state = random_number)

In [ ]:
 def get_image_array(filename):
        image_string = tf.io.read_file(filename)
        image = tf.image.decode_jpeg(image_string)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = image/255.0
        return image


In [ ]:
x_train_ = []
y_train_ = []
x_val_ = []
y_val_ = []

for pair, row in train_df.iterrows():
    image1 = get_image_array(row["Path 1"])
    image2 = get_image_array(row["Path 2"])
    x_train_.append([image1,image2])
    y_train_.append(int(row["Label"]))
    
for pair, row in val_df.iterrows():
    image1 = get_image_array(row["Path 1"])
    image2 = get_image_array(row["Path 2"])
    x_val_.append([image1,image2])
    y_val_.append(int(row["Label"]))
    
x_train = np.array(x_train_)
y_train = np.array(y_train_)
x_val = np.array(x_val_)
y_val = np.array(y_val_)


In [ ]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = tf.reduce_sum(tf.square(tf.subtract(x, y)), axis=1, keepdims=True)
    return tf.sqrt(tf.maximum(sum_square, K.epsilon()))

In [ ]:
import os
os.environ["TFHUB_CACHE_DIR"] = "C:/Users/prokopj/"
hub_extractor = "https://tfhub.dev/google/bit/m-r50x1/1"



In [ ]:
feature_extractor_layer = hub.KerasLayer(
    hub_extractor,
    input_shape=(224, 224, 3),
    trainable=False)


In [ ]:
def initialize_base_network():
    input = Input(shape=(224,224,3))
    x = feature_extractor_layer(input)
    x = Dense(128, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    return Model(inputs=input, outputs=x)

In [ ]:
base_network = initialize_base_network()

In [ ]:
input_a = Input(shape=(224,224,3), name="left_input")
vect_output_a = base_network(input_a)
input_b = Input(shape=(224,224,3), name="right_input")
vect_output_b = base_network(input_b)
output = Lambda(euclidean_distance, name="output_layer")([vect_output_a, vect_output_b])
model = Model([input_a, input_b], output)
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
def contrastive_loss_with_margin(margin):
    def contrastive_loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        square_pred = tf.square(y_pred)
        margin_square = tf.square(tf.maximum(margin - y_pred, 0))
        return (y_true * square_pred + (1 - y_true) * margin_square)    
    return contrastive_loss


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights = True)

In [ ]:
%%time
model.compile(loss=contrastive_loss_with_margin(1.0), optimizer=Adam())
history = model.fit(
    [x_train[:,0],
    x_train[:,1]],
    y_train,
    batch_size=16,
    verbose=1,
    validation_data=([x_val[:,0],x_val[:,1]],y_val), 
    epochs=20,
    callbacks=[callback]
)



In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')

plt.xlabel('Epoch')
plt.legend()
plt.grid(True)


In [ ]:
left_image = x_val[:,0]
left_image_feature_vectors = base_network.predict(left_image)


In [ ]:
right_image = x_val[:,1]
right_image_feature_vectors = base_network.predict(right_image)

In [ ]:
positive_left_image_feature_vectors = left_image_feature_vectors[0::2]
negative_left_image_feature_vectors = left_image_feature_vectors[1::2]

positive_right_image_feature_vectors = right_image_feature_vectors[0::2]
negative_right_image_feature_vectors = right_image_feature_vectors[1::2]

In [ ]:
positive_distances = euclidean_distance((positive_left_image_feature_vectors, positive_right_image_feature_vectors)).numpy().flatten()
pd.Series(positive_distances).describe()


In [ ]:
negative_distances = euclidean_distance((negative_left_image_feature_vectors, negative_right_image_feature_vectors)).numpy().flatten()
pd.Series(negative_distances).describe()


In [ ]:
border = abs(np.mean(pd.Series(positive_distances)) + np.mean(pd.Series(negative_distances)))/2
print(border)

In [ ]:
def compute_accuracy(y_true, y_pred):
    # 1 for the same - distance is smaller than 0.5, 0 for the different
    y_true = tf.cast(y_true, tf.float32)
    pred = y_pred.flatten() < 0.5
    return np.mean(pred == y_true)


In [ ]:
y_pred_test = model.predict([x_val[:,0], x_val[:,1]])
test_accuracy = compute_accuracy(y_val, y_pred_test)
print(f'Test accuracy: {test_accuracy*100:.2f}%')

In [ ]:
photos = pd.read_csv("pairsteam.csv")

In [ ]:
images_dir_team = 'C:/Users/prokopj/team/'

In [ ]:
photos['path'] = images_dir_team + photos['path']

In [ ]:
def get_image_array(filename):
        image_string = tf.io.read_file(filename)
        image = tf.image.decode_jpeg(image_string)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = image/255
        return image   

In [ ]:
def comparing_photos(label):
    same_photos = []
    for photo, row in photos.iterrows():
        if row["label"] == label:
            image = get_image_array(row["path"])
            same_photos.append(image)
    euclidian_distance = model.predict([np.expand_dims(same_photos[0], axis=0),np.expand_dims(same_photos[1], axis=0)])
    print(euclidian_distance)
    team_dict = {1:"Petr", 2:"Martin",3:"Dominika",4:"Karel",5:"Jonáš"}
    print(team_dict[label])
    x = 0
    if euclidian_distance<border:
        print("Photos are the same") 
        return (x)
    else:
        print("Photos are not the same")
        return (x+1)

In [ ]:
def iteration_through_pairs(n):
    same = 0
    dif = 0
    for i in range(n):
        number = comparing_photos(i+1)
        if number == 0:
            same = same + 1
        if number == 1: 
            dif = dif +1
    print(f"The model classifiated pictures of team members with given accuracy: {same/(same+dif)}")

In [ ]:
iteration_through_pairs(5)